In [1]:
### Consensus analysis using mean Z score
import numpy
import scipy.stats
import matplotlib.pyplot as plt

# from Tom:

Following on our chat, I realise that my notes were missing some indices. Here’s a corrected version:

    Y_i: N-vector of T scores (or whatever) at voxel i
    Var(Y_i) =  Sigma_i

where Sigma_i is the NxN covariance matrix. But to be practical, we need to assume *common* variance, and a *global* correlation:

    Var(Y_i) =  sigma^2 Q

Where sigma is the (scalar) variance for whole image, Q the common NxN correlation (*not* covariance)

Then the average is 

    bar{Y_i} = X’ Y_i / N

where X is a column of ones and

    Var(bar{Y_i}) =  sigma^2   X’ Q X / N^2

So then the T test is 

    T_i = bar(Y_i) / sqrt(Var(bar{Y_i}))

I don't think the variance should be estimated over submissions/teams, but if you were to do so you could do it at each voxel as:

    Y_i’ R Y_i / tr(RQ)

then the effective DF is as you say,

    v = tr(RQ)^2 / tr(RQRQ)

But you could also use the naive estimate

    hat{sigma^2_i} = Y_i’ R Y_i / (N-1)
    
Finally, this will give a completely noramlised test statistic... i.e. a T_i image that is variance 1.  If we wish to retain the average variance of the various test statistics, we simply need to drop sigma^2 from the definition of Var(bar{Y_i}).

In [129]:
def t_corr(y,res_mean=None,res_var=None,Q=None):
    """
    perform a one-sample t-test on correlated data
    y = data (n observations X n vars)
    Q = "known" correlation across observations (use empirical correlation based on maps)
    """
    
    # Jeanette:
    # This paper calculates the df for an F-test, so the chisquare bit we need is in there.  Your t-statistic will come from
    # X = column of 1's (design matrix)

    if len(y.shape)==1:
        y = y[:,numpy.newaxis]
    assert y.shape[1]==1
    
    npts = y.shape[0]
    X = numpy.ones((npts,1))

    if res_mean is None:
        res_mean = 0

    if res_var is None:
        res_var = 1
  
    if Q is None:
        Q = numpy.eye(npts)

    # R = I{n} - X(X'X)^{-1}X'
    R = numpy.eye(npts) - X.dot(numpy.linalg.inv(X.T.dot(X))).dot(X.T)

    
    VarMean = res_var * X.T.dot(Q).dot(X) / npts**2

    # T  =  mean(y,0)/s-hat-2
    # use diag to get s_hat2 for each variable 
    T = (numpy.mean(y,0)-res_mean)/numpy.sqrt(VarMean)*numpy.sqrt(res_var) + res_mean

    # degrees of freedom = v = tr(RQ)^2/tr(RQRQ)
    df = (numpy.trace(R.dot(Q))**2)/numpy.trace(R.dot(Q).dot(R).dot(Q))
    p = 1 - scipy.stats.t.cdf(T,df=df)
    return(T,df,p)

In [130]:
npts = 36
nvars = 10
nruns=1000
alpha=.05
mu=0
# simulate independent case
pvals= numpy.zeros((nruns,nvars))

for i in range(nruns):
    y = numpy.random.randn(npts,nvars)
    result = t_corr(y)
    pvals.append(result[2].tolist())
pvals_mtx = numpy.array(pvals)   
numpy.mean(pvals_mtx<=0.05)   # If p-values valid/nominal, 5% should be below 0.05


0.0429

In [131]:
npts = 36
nvars = 10
nruns=100
rho=0.9

# now simulate correlated data

def mk_CS_Cov(npts,rho):
    Cov = (1-rho)*numpy.identity(npts)+rho*numpy.ones([npts,npts])
    return(Cov)
            
Q = mk_CS_Cov(npts,rho)

def mk_correlated_data(npts,nvars,Cov):
    
    pvals = []
    for i in range(nruns):
        y = numpy.random.multivariate_normal(numpy.zeros(npts),Cov,nvars).T
    return(y)


In [133]:
# Apply simulation 'right' way, telling t_corr about correlation
pvals = []
for i in range(nruns):
    y = mk_correlated_data(npts,nvars,Q)
    result = t_corr(y,0,1,Q)
    pvals.append(result[2].tolist())
pvals_mtx = numpy.array(pvals)   
numpy.mean(pvals_mtx<=0.05)   # If p-values valid/nominal, 5% should be below 0.05


0.048

In [134]:
# Apply simulation 'wrong' way, telling t_corr about correlation
pvals = []
for i in range(nruns):
    y = mk_correlated_data(npts,nvars,Q)
    result = t_corr(y)
    pvals.append(result[2].tolist())
pvals_mtx = numpy.array(pvals)   
numpy.mean(pvals_mtx<=0.05)   # If p-values valid/nominal... but you'll surely find it higher


0.412

In [137]:
# Now try a *non* null simulation
Mn=2
Sd=1.5
Tvals = []
for i in range(nruns):
    y = Mn+Sd*mk_correlated_data(npts,nvars,Q)
    result = t_corr(y,2,1.5,Q)
    Tvals.append(result[0].tolist())
Tvals_mtx = numpy.array(Tvals)
print(numpy.mean(Tvals_mtx),numpy.std(Tvals_mtx))   # Should match Mn & Sd


1.987010776746394 1.4961018870404887
